In [1]:
%pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Preformatted
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle # ParagraphStyle import
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import traceback

In [3]:
# 폰트 파일이 있는 디렉토리 경로
font_directory = '/System/Library/Fonts/Supplemental'
korean_font_filename = 'Arial Unicode.ttf'

# 완전한 폰트 파일 경로 생성
full_korean_font_path = os.path.join(font_directory, korean_font_filename)

# ReportLab에 등록할 폰트 이름
REPORTLAB_FONT_NAME = 'Arial Unicode'

try:
    # 폰트 등록
    pdfmetrics.registerFont(TTFont(REPORTLAB_FONT_NAME, full_korean_font_path))

    # 한글 폰트가 올바르게 작동하도록 폰트 패밀리 매핑
    # 이렇게 해야 'Normal' 스타일 등을 사용할 때 한글이 깨지지 않습니다.
    pdfmetrics.registerFontFamily(REPORTLAB_FONT_NAME,
                                  normal=REPORTLAB_FONT_NAME,
                                  bold=REPORTLAB_FONT_NAME, # 또는 REPORTLAB_FONT_NAME_BOLD
                                  italic=REPORTLAB_FONT_NAME, # 이탤릭이 필요한 경우 (없다면 normal과 동일하게)
                                  boldItalic=REPORTLAB_FONT_NAME # 볼드 이탤릭이 필요한 경우
                                )

    print(f"폰트 '{korean_font_filename}'이(가) '{REPORTLAB_FONT_NAME}' 이름으로 성공적으로 등록되었습니다.")

except Exception as e:
    traceback.print_exc()
    exit() # 폰트 등록 실패 시 프로그램 종료

폰트 'Arial Unicode.ttf'이(가) 'Arial Unicode' 이름으로 성공적으로 등록되었습니다.


In [4]:
def create_code_pdf(source_folder, output_pdf_path, exclude_dirs=None):
    if not os.path.isdir(source_folder):
        print(f"오류: '{source_folder}'는 유효한 폴더가 아닙니다.")
        return False

    doc = SimpleDocTemplate(output_pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # 스타일 정의 시 등록된 한글 폰트 이름 사용
    # 원본 스타일을 직접 수정하는 대신, ParagraphStyle을 사용하여 복사본을 만들어 수정합니다.
    title_style = ParagraphStyle(name='TitleStyle', parent=styles['h1'])
    title_style.fontName = REPORTLAB_FONT_NAME

    filepath_style = ParagraphStyle(name='FilePathStyle', parent=styles['h3'])
    filepath_style.fontName = REPORTLAB_FONT_NAME

    code_list_style = ParagraphStyle(name='CodeListStyle', parent=styles['Normal'])
    code_list_style.fontName = REPORTLAB_FONT_NAME
    code_list_style.fontSize = 10
    code_list_style.leading = 12 # 줄 간격

    # 'Code' 스타일이 없으면 새로 정의
    # Preformatted는 'Code' 스타일이 없으면 'Normal'을 기본으로 사용합니다.
    if 'Code' not in styles:
        code_style = ParagraphStyle(name='CodeCustomStyle', parent=styles['Normal'])
        code_style.fontName = REPORTLAB_FONT_NAME
        code_style.fontSize = 7
        code_style.leading = 8 # 코드 줄 간격
        # code_style.tabwidth = 4 * code_style.fontSize # 탭 너비를 4칸 공백으로 설정 (더 이상 필요 없음)
    else:
        code_style = ParagraphStyle(name='CodeCustomStyle', parent=styles['Code'])
        code_style.fontName = REPORTLAB_FONT_NAME
        code_style.fontSize = 7
        code_style.leading = 8 # 코드 줄 간격
        # code_style.tabwidth = 4 * code_style.fontSize # 탭 너비를 4칸 공백으로 설정 (더 이상 필요 없음)


    all_files_to_include = [] # PDF에 포함될 파일 경로를 저장할 리스트

    # 제외할 디렉토리 목록을 설정
    excluded_directories = [os.path.normpath(d) for d in (exclude_dirs if exclude_dirs is not None else [])]

    # 모든 PHP 및 JSON 파일의 경로를 먼저 수집 (제외 디렉토리 고려)
    found_files = False
    for root, dirs, files in os.walk(source_folder):
        relative_root = os.path.relpath(root, start=source_folder)
        
        is_excluded_root = False
        for ex_dir in excluded_directories:
            if relative_root == ex_dir or relative_root.startswith(ex_dir + os.sep):
                is_excluded_root = True
                break
        
        if is_excluded_root and relative_root != '.':
            # 현재 root가 제외 대상이면 이 폴더와 하위 폴더는 더 이상 탐색하지 않음
            dirs[:] = [] 
            continue

        for file in files:
            if file.endswith(('.php', '.json')):
                filepath = os.path.join(root, file)
                relative_filepath = os.path.relpath(filepath, start=source_folder)
                
                # 파일 경로가 제외될 디렉토리 내에 있는지 확인
                should_exclude_file = False
                for ex_dir in excluded_directories:
                    if relative_filepath.startswith(ex_dir + os.sep) or relative_filepath == ex_dir:
                        should_exclude_file = True
                        break
                
                if not should_exclude_file:
                    found_files = True
                    all_files_to_include.append(filepath)

    if not found_files:
        story.append(Paragraph("지정된 폴더에서 PHP 또는 JSON 파일을 찾을 수 없습니다.", styles['Normal']))
        print("경고: PDF에 추가할 PHP 또는 JSON 파일이 발견되지 않았습니다.")
        try:
            doc.build(story)
            print(f"PDF 파일이 성공적으로 생성되었습니다: {output_pdf_path} (포함된 파일 없음)")
            return True
        except Exception as e:
            print(f"PDF 생성 중 오류 발생: {e}")
            traceback.print_exc()
            return False

    # --- 첫 페이지: 전체 경로 목록 추가 ---
    story.append(Paragraph("PDF에 포함된 파일 경로 목록", title_style))
    story.append(Spacer(1, 0.2 * inch))

    all_files_to_include.sort()
    for filepath in all_files_to_include:
        relative_filepath = os.path.relpath(filepath, start=source_folder)
        story.append(Paragraph(f"- {relative_filepath}", code_list_style))

    story.append(PageBreak())

    # --- 두 번째 페이지부터: 각 파일 내용 추가 ---
    for filepath in all_files_to_include:
        relative_filepath = os.path.relpath(filepath, start=source_folder)

        story.append(Paragraph(f"--- 파일 경로: {relative_filepath} ---", filepath_style))
        story.append(Spacer(1, 0.1 * inch))

        try:
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                
                # !!! 중요: 탭 문자를 4개의 공백으로 대체 (이 부분 추가/수정) !!!
                # 이렇게 하면 폰트나 ReportLab 설정과 무관하게 탭 간격이 일관됩니다.
                content = content.replace('\t', '    ') # 탭을 4개의 공백으로 대
                
                code_paragraph = Preformatted(content, code_style)
                story.append(code_paragraph)
                story.append(Spacer(1, 0.3 * inch))
        except Exception as e:
            story.append(Paragraph(f"파일을 읽는 중 오류 발생: {filepath} - {e}", styles['Normal']))
            story.append(Spacer(1, 0.2 * inch))
            traceback.print_exc()

    try:
        doc.build(story)
        print(f"PDF 파일이 성공적으로 생성되었습니다: {output_pdf_path}")
        return True
    except Exception as e:
        print(f"PDF 생성 중 오류 발생: {e}")
        traceback.print_exc()
        return False

In [5]:
exclude_dirs_list = ["vendor"]

source_directory = ".".strip()
output_base_name = "flexphp-banana".strip()

# PDF 파일명 설정
output_pdf_filename = f"{output_base_name}.pdf"
create_code_pdf(source_directory, output_pdf_filename, exclude_dirs=exclude_dirs_list)

print("\n--- 작업 완료 ---")

PDF 파일이 성공적으로 생성되었습니다: flexphp-banana.pdf

--- 작업 완료 ---
